<a href="https://colab.research.google.com/github/GrupoTuring/NLP-fb-chatbot/blob/main/projeto_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.5MB 5.9MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
     |████████████████████████████████| 2.9MB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2fa1867f057574a56029741416bc44f1d479734d3d90a6a7c802512519d56a6e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import AutoTokenizer  
from transformers import AutoModel 
import torch
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

class QuestionEmbeddings():
    def __init__(self, question_path, greeting, no_answer, similarity_threshold=0.69, count_stops=False, media=True):
        self.no_answer = no_answer
        self.similarity_threshold = similarity_threshold
        self.greeting = greeting
        self.count_stops = count_stops
        self.media = media
        self.tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
        self.model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
        self.stopwords = nltk.corpus.stopwords.words('portuguese')
        perguntas_frequentes_int = pd.read_excel(question_path)
        self.perguntas_frequentes = self.get_database_embs(perguntas_frequentes_int)

    def get_database_embs(self, perguntas_frequentes):
        embs = []
        for i, row in perguntas_frequentes.iterrows():
            emb = self.get_sentence_embs(row['PERGUNTAS'])
            embs.append(emb)
        perguntas_frequentes['Sentence Embedding'] = embs
        return perguntas_frequentes

    def get_embs_bertinbau(self, frase):
        input_ids = self.tokenizer.encode(frase, return_tensors='pt')
        with torch.no_grad():
            outs = self.model(input_ids)
            encoded = outs[0][0, 1:-1] 
        return encoded

    def get_sentence_embs(self, frase):
        frase_embs = self.get_embs_bertinbau(frase.lower()).numpy()
        frase_embs_considerados = []
        palavras = word_tokenize(frase.lower())
        for palavra, embedding in zip(palavras, frase_embs):
            if not(self.count_stops):
                if not(palavra in self.stopwords):
                    frase_embs_considerados.append(embedding)
            else:
                frase_embs_considerados.append(embedding)
        frase_embs_considerados = np.array(frase_embs_considerados)
        if self.media:
            final = np.mean(frase_embs_considerados, axis=0)
        else:
            final = np.sum(frase_embs_considerados, axis=0)
        return final
    
    def semelhanca_cossenos(self, a,b):
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    
    def get_most_similar_phrase(self, frase):
        mais_similar = ""
        maior_score_similaridade = -999

        sentence_emb = self.get_sentence_embs(frase)
        for i, row in self.perguntas_frequentes.iterrows():
            similaridade = self.semelhanca_cossenos(sentence_emb,row['Sentence Embedding'])
            if similaridade > maior_score_similaridade:
                maior_score_similaridade = similaridade
                mais_similar = row['PERGUNTAS']
        return mais_similar, maior_score_similaridade
    
    def get_response(self, frase):
        question, similaridade = self.get_most_similar_phrase(frase)
        anwser = self.perguntas_frequentes[self.perguntas_frequentes['PERGUNTAS']  ==  question]['RESPOSTAS'].values[0]
        print(f"Pergunta mais similar na base de dados: \n {question}\nsimilaridade = {similaridade*100}%")
        if similaridade < self.similarity_threshold:
            anwser = self.no_answer
        full_answer = self.greeting  + anwser
        return full_answer


In [ ]:
gret = "Olá! Obrigado por entrar em contato!"
noans = "Em breve um de nossos membros te responderá!"
qe = QuestionEmbeddings("/content/q_and_a.xlsx", gret, noans)

In [ ]:
qe.get_response("quando é o processo seletivo?")

Pergunta mais similar na base de dados: 
 Quando abre o Processo Seletivo?
similaridade = 94.71486806869507%


'Olá! Obrigado por entrar em contato!\nO processo seletivo do Grupo Turing apenas uma vez por ano, no inicio/meio do primeiro semestre. Divulgamos nas nossas redes sociais!'

In [ ]:
qe.get_response("como é o ps?")

Pergunta mais similar na base de dados: 
 Poderia me explicar como funciona o PS?
similaridade = 75.28309226036072%


'Olá! Obrigado por entrar em contato!\nAs regras específicas do nosso processo seletivo serão informadas quando iniciarmos a divulgação. Mas, em geral, contamos tanto com etapas técnicas, como entrevistas e dinâmicas'

In [ ]:
qe.get_response("vocês ensinam python?")

Pergunta mais similar na base de dados: 
 Vocês oferecem curso basico de Python?
similaridade = 78.89406085014343%


'Olá! Obrigado por entrar em contato!\nNão oferecemos curso de Python ou de qualquer outra linguagem de programação. Nosso grupo é voltado para pesquisa, aplicações de projetos e realizações de eventos. Não realizamos aulas externas.'

In [ ]:
qe.get_response("vcs poderiam divulgar uma vaga de estágio?")

Pergunta mais similar na base de dados: 
 Eu queria saber se vocês tem algum local especifico para fazer divulgação de vagas de emprego
similaridade = 80.09474277496338%


'Olá! Obrigado por entrar em contato!\nVocê pode utilizar nosso servidor no Discord para divulgar vagas de emprego - https://discord.gg/26RGmBS '

In [ ]:
qe.get_response("o pãozinho do sibuya tá disponível pra umas bitocas?")

Pergunta mais similar na base de dados: 
 O Sibuya está solteiro?
similaridade = 72.02653288841248%


'Olá! Obrigado por entrar em contato!\nSolteríssimo, só vem'

In [ ]:
qe.get_response("quanto custa essa escola?")

Pergunta mais similar na base de dados: 
 Quanto custa esta escola?
similaridade = 94.44549679756165%


'Olá! Obrigado por entrar em contato!\nNão somos uma escola'

In [ ]:
qe.get_response("sou do doutorado, posso entrar no grupo?")

Pergunta mais similar na base de dados: 
 Alunos da pós Graduação, mestrado e doutorado podem fazer parte do grupo?
similaridade = 79.5280933380127%


'Olá! Obrigado por entrar em contato!\nalunos da pós Graduação, mestrado e doutorado podem fazer parte do grupo'

In [ ]:
qe.get_response("voces fazem cursos de programação?")

Pergunta mais similar na base de dados: 
 Vocês oferecem curso basico de Python?
similaridade = 76.1618971824646%


'Olá! Obrigado por entrar em contato!\nNão oferecemos curso de Python ou de qualquer outra linguagem de programação. Nosso grupo é voltado para pesquisa, aplicações de projetos e realizações de eventos. Não realizamos aulas externas.'

In [ ]:
qe.get_response("Receberia salário no grupo?")

Pergunta mais similar na base de dados: 
 Ao participar do grupo, recebe-se algum salário, bolsa ou compensação financeira?
similaridade = 70.029217004776%


'Olá! Obrigado por entrar em contato!\nNão há nenhum tipo de contrapartida financeira para os membros do grupo. O trabalho é 100% voluntário'

In [ ]:
qe.get_response("como funciona o grupo?")

Pergunta mais similar na base de dados: 
 Como posso conhecer o grupo?
similaridade = 71.89542055130005%


'Olá! Obrigado por entrar em contato!\nPara conhecer o grupo, veja nosso Instagram @grupoturing.usp, nossos posts no medium (turing-talks), conheça nosso site (grupoturing.com.br) e entre na nossa comunidade no Discord - https://discord.gg/26RGmBS '

In [ ]:
qe.get_response("quero entrar em contato com o grupo")

Pergunta mais similar na base de dados: 
 Como posso entrar em contato com vocês?
similaridade = 76.11371874809265%


'Olá! Obrigado por entrar em contato!\nPode mandar a mensagem por aqui mesmo, mas se quiser algo mais formal, pode nos enviar um e-mail no grupoturing.usp@gmail.com'

In [ ]:
qe.get_response("Vocês tem conteudo de processamento de linguagem natural (pnl)?")

Pergunta mais similar na base de dados: 
 Me interesso muito por Processamento de Linguagem Natural, NLP ou PLN e gostaria de falar com vocês sobre isso
similaridade = 80.10140061378479%


'Olá! Obrigado por entrar em contato!\nTemos alguns textos no nosso medium sobre Processamento de Linguagem Natural! Se tiver alguma dúvida em específico, sinta-se à vontade para enviar uma mensagem no nosso servidor no discord ( https://discord.gg/26RGmBS )'

In [ ]:
qe.get_response("sou da pós graduação, posso entrar no grupo?")

Pergunta mais similar na base de dados: 
 Alunos da pós Graduação, mestrado e doutorado podem fazer parte do grupo?
similaridade = 87.89551854133606%


'Olá! Obrigado por entrar em contato!\nalunos da pós Graduação, mestrado e doutorado podem fazer parte do grupo'

In [ ]:
qe.get_response("quando acaba o coronga?")

Pergunta mais similar na base de dados: 
 O Sibuya está solteiro?
similaridade = 49.07633662223816%


'Olá! Obrigado por entrar em contato!\nEm breve um de nossos membros te responderá!'